# Scrape

### 1. Setup

The setup serves for:
- Importing libraries.
- Speficying utility functions.
- Setting up the chrome webdriver.
- Defining scraping constants.

In [13]:
# imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import random

In [42]:
# utils

def random_seconds():
    return random.randint(1, 2)

def sleep():
    time.sleep(random_seconds())

In [17]:
# set up ChromeDriver

options = Options()
options.add_argument('--headless')  # save GUI rendering time
options.add_argument('--disable-gpu')  # prevents errors in headless mode

service = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

# open base page
BASE_URL = 'https://www.migros.ch/en'
driver.get(BASE_URL)
time.sleep(5)

In [18]:
# constants

# go to subcategory
SUBCAT_XPATH = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/mo-category-level-1-children-categories/div/ul/li[{}]/div/div/a'

# go to page
TEMPLATE_URL = '?page={}'

# hand picked from base url
CATEGORIES = [
    'fruits-vegetables',
    'bread-pastries-breakfast',
    'pasta-condiments-canned-food',
    'snacks-sweets',
    'frozen-food',
    'drinks-coffee-tea',
    'wine-beer-spirits'
]

### 2. Scrape links

Considerations and steps:
- Since all products have pages have the same template,
it is convenient separating the link retrieval and the product specifics scraping.
- In this first step we navigate to category, sub-category, and scrape all products links from the subcategories subpages.
- All links are saved a dataframe, and then permanently saved to csv. 
- The same dataframe will be used to store the product specifics in the next step.

In [9]:
# list of dictionaries 
data = []

# big giant massive loop
for cat in CATEGORIES:
    url = BASE_URL + '/category/' + cat
    driver.get(url) 

    sleep()

    # dynamically find subcategories (li[n] elements)
    subcategory_urls = []
    li_index = 1

    while True:
        sleep()
        subcat_xpath = SUBCAT_XPATH.format(li_index)
        try:
            subcategory = driver.find_element(By.XPATH, SUBCAT_XPATH)
            subcategory_url = subcategory.get_attribute('href')
            subcategory_urls.append(subcategory_url)

            print(f"Found subcategory link: {subcategory_url}")
            li_index += 1

        except NoSuchElementException:
            print("No more subcategory elements found.")
            break

    print(f"Subcategories found: {subcategory_urls}")

    for subcat_url in subcategory_urls:
        driver.get(subcat_url) 
        sleep()

        # extract products and go to the next page
        scrape_url = subcat_url + TEMPLATE_URL
        print(f"Scraping url: {scrape_url}")
        product_urls = []
        page = 1
        while True:
            sleep()
            url = scrape_url.format(page)
            driver.get(url)
            print(f"Scraping url: {url}")
            
            try:
                product_list = driver.find_elements(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/div/mo-items-display/div/ul/li/article/div/div[1]/a[1]')
                if not product_list:
                    print(f"No more products found on page {page}. Exiting.")
                    break
                
                for product in product_list:
                    try:
                        product_url = product.get_attribute('href')
                        product_urls.append(product_url)
                    except Exception as e:
                        print(f"Error processing product: {e}")
                        continue
                
                print(f"Finished scraping page {page}.")
                page += 1
            
            except NoSuchElementException:
                print(f"Error on page {page}, stopping.")
                break

        # store
        current_data = {
            'category': cat,
            'sub_category': subcat_url.split('/')[-1],
            'url': product_urls
        }
        data.append(current_data)

        # report 
        print(f"Finished scraping subcategory {subcat_url}")
        print(f"Data: {current_data}")

Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/fruits
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/vegetables
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/root-vegetables
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/salad
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/fresh-herbs-spices
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/ready-to-use
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/vitamin-baskets
No more subcategory elements found.
Subcategories found: ['https://www.migros.ch/en/category/fruits-vegetables/fruits', 'https://www.migros.ch/en/category/fruits-vegetables/vegetables', 'https://www.migros.ch/en/category/fruits-vegetables/root-vegetables', 'https://www.migros.ch/en/category/fruits-vegetables/salad', 'https://www.migros.ch/en/category/fruits-vegetables/fresh-her

In [ ]:
# convert to dataframe

df = pd.DataFrame(data)
df = df.explode('url')
df.to_csv('products.csv', index=False)

### 3. Scrape product information

Considerations and steps:
- 

In [ ]:
# Close the browser
driver.quit()

I now have a csv with all product links. I now need to do a second scraper and for each product extract following information:

- name:
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div[2]/mo-product-detail-core-information/div/div[3]/mo-product-detail-title/h1
example: <h1 _ngcontent-website-js-c3089092147="" id="product-detail-100025208-title" data-cy="product-detail-100025208-title" class="ng-star-inserted"> Apples · SweeTango </h1>

- product weight:
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div[2]/mo-product-detail-core-information/div/div[4]/mo-product-detail-quantity/span/text()
example: 700–950 g 

- price:
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div[2]/mo-product-detail-core-information/div/div[2]/mo-product-detail-price/span/span
example: <span _ngcontent-website-js-c1288947006="" class="actual" id="100025208-original-price" data-cy="100025208-original-price"> 4.90 </span>

- price/100g:
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[1]/div[2]/div[2]/mo-product-detail-core-information/div/div[4]/mo-product-detail-price-quantity/span/span
example: <span _ngcontent-website-js-c816549211="" id="product-detail-100025208-price-unit"> 0.60/100g </span>

- Most important attributes (list of mos, with attribute and value):
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[1]/mo-product-detail-main-information/dl/mo-product-information-item[2]
attribute example: /html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[1]/mo-product-detail-main-information/dl/mo-product-information-item[2]/dt
value example: /html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[1]/mo-product-detail-main-information/dl/mo-product-information-item[2]/dd

- Table with nutritional values:
/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[2]/mo-product-detail-nutrients-information/table/tbody
energy: /html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[2]/mo-product-detail-nutrients-information/table/tbody/tr[1]/td[1]/div/span
energy value: /html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[2]/mo-product-detail-nutrients-information/table/tbody/tr[1]/td[2]



In [44]:
df = pd.read_csv('data/products.csv')

from bs4 import BeautifulSoup

# Initialize a list to store all the product data
product_data = []

counter = 0
total = len(df)

# Loop through each product URL
for index, row in df.iterrows():
    product_url = row['url']
    driver.get(product_url)
    sleep()  # Wait for the page to load

    try:
        # Extract the product name
        product_name = driver.find_element(By.CSS_SELECTOR, 'div.core-product-title h1').text

        # Extract the product price
        try:
            product_price = driver.find_element(By.CSS_SELECTOR, 'span.actual').text
        except Exception:
            product_price = "N/A"
            print(f"Product price not found for {product_name}")

        # Extract the product weight
        try:
            product_weight = driver.find_element(By.CSS_SELECTOR, 'span.weight-priceUnit').text
        except Exception:
            product_weight = "N/A"
            print(f"Product weight not found for {product_name}")

        # Extract the price per unit (e.g., per 100g)
        try:
            price_per_unit = driver.find_element(By.CSS_SELECTOR, 'span.listMode-priceUnit').text
        except Exception:
            price_per_unit = "N/A"
            print(f"Price per unit not found for {product_name}")

    
        ########
        # Initialize dictionary to store nutritional information
        nutrition_info = {}
        try:
            # Extract the full HTML of the table containing nutritional information
            table_html = driver.find_element(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[2]/mo-product-detail-nutrients-information/table').get_attribute('outerHTML')

            # Use BeautifulSoup to parse the HTML
            soup = BeautifulSoup(table_html, 'html.parser')

            # Find all rows in the table
            rows = soup.find('tbody').find_all('tr')

            # Loop through the rows and extract data
            for row in rows:
                try:
                    # Extract the nutrient name from the first column (td[1])
                    nutrient_name = row.find_all('td')[0].get_text(strip=True)

                    # Extract the nutrient value from the second column (td[2])
                    nutrient_value = row.find_all('td')[1].get_text(strip=True)

                    # Store the nutrient name and value in the dictionary
                    nutrition_info[nutrient_name] = nutrient_value

                except Exception as e:
                    print(f"Error processing row: {e}")
                    continue

        except Exception as e:
            print(f"Error extracting table.")
        ########

        # Store the scraped data in the list
        current_data = {
            'Product URL': product_url,
            'Name': product_name,
            'Weight': product_weight,
            'Price': product_price,
            'Price per 100g': price_per_unit,
            'Nutritional Information': nutrition_info
        }
        product_data.append(current_data)

        counter += 1

        print(f"{counter}/{total}: {current_data}")

    except Exception as e:
        print(f"Error processing product at {product_url}: {str(e)}")
        continue

1/9602: {'Product URL': 'https://www.migros.ch/en/product/265680802400', 'Name': 'Migros Bio · blueberries', 'Weight': '250g', 'Price': '7.90', 'Price per 100g': '3.16/100g', 'Nutritional Information': {'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g', 'of which saturates': '0 g', 'Carbohydrate': '6 g', 'of which sugars': '6 g', 'Fibre': '5 g', 'Protein': '0.6 g', 'Salt': '< 0.01 g'}}
Error extracting table.
2/9602: {'Product URL': 'https://www.migros.ch/en/product/264601302100', 'Name': 'Dates', 'Weight': '200g', 'Price': '5.95', 'Price per 100g': '2.98/100g', 'Nutritional Information': {}}
3/9602: {'Product URL': 'https://www.migros.ch/en/product/264500313200', 'Name': 'Avocado', 'Weight': '', 'Price': '2.80', 'Price per 100g': '2.80/Piece', 'Nutritional Information': {'Energy': '643 kJ (156 kcal)', 'Fat': '14 g', 'of which saturates': '2 g', 'Carbohydrate': '1.8 g', 'of which sugars': '0.7 g', 'Fibre': '7 g', 'Protein': '1.8 g', 'Salt': '0 g'}}
4/9602: {'Product URL': 'https://www.migro

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(product_data)
df.to_csv('product_data.csv', index=False)

df.head(10)